In [ ]:
 import os
import numpy as np
from tqdm import tqdm_notebook, tnrange
from skimage.util.shape import view_as_windows
from keras.preprocessing import image
from skimage.util import crop
from PIL import ImageDraw, ImageFont, Image
!pip install image_slicer
import image_slicer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt

%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/squeezenet_final.h5" "/content/squeezenet.h5"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/u-net_patch2.h5" "/content/u-net.h5"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/smoke_test.zip" "/content/test_set.zip"
#!unzip test_set.zip
%cd /content

seg_model = load_model("u-net.h5")
class_model = load_model("squeezenet.h5")


     |████████████████████████████████| 2.2MB 14.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


/content


In [ ]:
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/squeezenet_final.h5" "/content/squeezenet.h5"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/u-net_patch2.h5" "/content/u-net.h5"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/smoke_test.zip" "/content/test_set.zip"
#!unzip test_set.zip
%cd /content

seg_model = load_model("u-net.h5")
class_model = load_model("squeezenet.h5")

In [ ]:
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/u-net_smoke5.h5" "/content/u-net.h5"

In [ ]:
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/fire_test.zip" "/content/test_set.zip"

cp: cannot stat '/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/fire_test.zip': No such file or directory


In [ ]:
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/fire test.zip" "/content/fire_test.zip"
%cp "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/smoke test.zip" "/content/smoke_test.zip"
!unzip fire_test.zip
!unzip smoke_test.zip

In [ ]:
def prepare_image (filename, size):
  img = image.load_img(filename)
  img = img.resize(size)
  img = np.asarray(img)
  x_img = np.expand_dims(img, axis=0)

  return img,x_img

def classification_predicition(image):
  fire = 0
  output = class_model.predict(image)
  if output[0][0] > class_threshold:
    fire = 1
  else:
    fire = 0
  return fire

input_size = (128,128)

In [ ]:
# Function for Predicition using patch sliding

def image_slicer_prediction (filename,n):
  square_h = 128*n
  square_w = 128*n

  img = image.load_img(filename)

  img = np.asarray(img)
  x_img = np.expand_dims(img, axis=0)

  size = img.shape
  #print ("original size:") 
  #print(size)
  squares_in_y = size[0]/square_h
  squares_in_x = size[1]/square_w
  #print(squares_in_x)
  #print(squares_in_y)

  squares_in_x = round(squares_in_x)
  squares_in_y = round(squares_in_x)
  target_size = (squares_in_y*square_h,squares_in_x*square_w)
  #print(target_size)

  img = image.load_img(filename,target_size)
  img = np.asarray(img)
  img_compressed = image.load_img(filename,(squares_in_x*128,squares_in_y*128))

  if squares_in_x > 1 and squares_in_y > 1:
    tiles = image_slicer.slice(filename,squares_in_y*squares_in_x, save=False)
    slide = 1
    #print ("sliding")
  else:
    slide = 0
    #print ("no sliding")

  i = 0
  if slide == 0:

    img, x_img = prepare_image(filename,input_size)
    fire = classification_predicition(x_img)
    
    if ( fire == 1):
      #print("fire in patch:" ,i)
      prediction = seg_model.predict(x_img/255)
      new_image = np.squeeze((prediction), axis=0)

      pred_bin = (prediction > seg_threshold).astype(np.uint8)
      pred_bin = np.squeeze((pred_bin), axis=0)

      prediction_image = array_to_img(new_image)
      prediction_image = prediction_image.resize([size[1],size[0]])
      bin_image = array_to_img(pred_bin)
      bin_image = bin_image.resize([size[1],size[0]])
      return (img_compressed,prediction_image,bin_image)

    else:
      new_image_x = Image.new('L', (128,128))
      new_image_x = new_image_x.resize([size[1],size[0]])
      return (img_compressed,new_image_x,new_image_x)
      

  if slide == 1:
    new_image_pred = Image.new('L', (squares_in_x*128,squares_in_y*128))
    new_image_pred_bin = Image.new('L', (squares_in_x*128,squares_in_y*128))
    for tile in tiles:
        img = tile.image
        img = img.resize((128,128))
        img = np.asarray(img)
        x_img = np.expand_dims(img, axis=0)
      
        fire = classification_predicition(x_img)
        
        if ( fire == 1):
          #print("fire in patch: " ,i)
          prediction = seg_model.predict(x_img/255)
          new_image = np.squeeze((prediction), axis=0)

          pred_bin = (prediction > seg_threshold).astype(np.uint8)
          pred_bin = np.squeeze((pred_bin), axis=0)
      
          new_image_pred.paste(array_to_img(new_image), ( (i*128 )%(squares_in_x*128), ( 128*int((i*128)/(squares_in_x*128)))) )
          new_image_pred_bin.paste(array_to_img(pred_bin), ( (i*128 )%(squares_in_x*128), ( 128*int((i*128)/(squares_in_x*128)))) )
        i += 1
    
  new_image_pred = new_image_pred.resize([size[1],size[0]])
  new_image_pred_bin = new_image_pred_bin.resize([size[1],size[0]])

  return (img_compressed,new_image_pred,new_image_pred_bin)

In [ ]:
# Predicition using Quad_Tree Method
import numpy as np
from skimage.util.shape import view_as_windows
from keras.preprocessing import image
from skimage.util import crop
from PIL import ImageDraw, ImageFont, Image
import image_slicer

def image_quad_prediction (filename, ratio, min_patch_size):

  orig_image = image.load_img(filename)
  
  square_image = orig_image.resize((128,128))
  square_image_class = orig_image.resize((192,192))

  orig_image = np.asarray(orig_image)
  square_image = np.asarray(square_image)

  square_image_class = np.asarray(square_image_class)
  
  x_orig_image = np.expand_dims(square_image, axis=0)
  x_square_image_class = np.expand_dims(square_image, axis=0)
  size = orig_image.shape
  #print(size)

  if ( class_threshold > 0):
    fire = class_model.predict(x_square_image_class)

  prediction = seg_model.predict(x_orig_image/255)

  pred_bin = (prediction > seg_threshold).astype(np.uint8)

  elements = size[0]*size[1]

  if ( (prediction>seg_threshold).sum() < ratio * elements) and ( size[0] > min_patch_size and size[1] > min_patch_size ):
    #print((prediction>threshold).sum() )
    #print ("sliding")
    new_image_pred = Image.new('L', (size[1],size[0]))
    new_image_bin = Image.new('L', (size[1],size[0]))

    tiles = image_slicer.slice(filename, 4 , save=False)
    i = 0
    for tile in tiles:
      tile.image.save("temp.png")
      result, pred_bin = image_quad_prediction ("temp.png", ratio, min_patch_size)
      if i == 0:
        new_image_pred.paste(result, ( 0 , 0))
        new_image_bin.paste(pred_bin, ( 0 , 0))
      elif i == 1:
        new_image_pred.paste(result, ( round(size[1]/2)-1,  0  ) )
        new_image_bin.paste(pred_bin, ( round(size[1]/2)-1,  0  ) )
      elif i == 2:
        new_image_pred.paste(result, ( 0 , round(size[0]/2)-1))
        new_image_bin.paste(pred_bin, ( 0 , round(size[0]/2)-1))
      else :
        new_image_pred.paste(result, ( round( size[1]/2)-1, round(size[0]/2)-1 )) #
        new_image_bin.paste(pred_bin, ( round( size[1]/2)-1, round(size[0]/2)-1 )) #

      i = i+1

  else:
    if ( class_threshold == 0 or fire[0][0] > class_threshold  ) :
      new_image = np.squeeze((prediction), axis=0)
      new_image = array_to_img(new_image)
      new_image = new_image.resize([size[1],size[0]])

      pred_bin = np.squeeze((pred_bin), axis=0)
      pred_bin = array_to_img(pred_bin)
      pred_bin = pred_bin.resize([size[1],size[0]])

    else:
      new_image = Image.new('L', (size[1],size[0]))
      pred_bin = Image.new('L', (size[1],size[0]))
    return new_image, pred_bin

  
  return new_image_pred, new_image_bin

In [ ]:
# Calculate Performance metrics between images 
def Calculate_Performance(i):
  masks_path = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/fire test/masks"
  result_folder = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/fire test/results"
  ids = next(os.walk(masks_path)) 
  ids = ids[2]
  #print("No. of images = ", len(ids))

  pos_iou_score_total = 0
  neg_iou_score_total  = 0
  TP = 0
  TN = 0
  FP = 0
  FN = 0

  total=len(ids)

  pos_iou_score = np.zeros(total)
  neg_iou_score = np.zeros(total)
  avg_iou_sing = np.zeros(total)

  for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
      # Load images
      mask_file = masks_path+"/"+id_
      result_file = result_folder+"/"+id_

      mask = image.load_img(mask_file)
      mask = np.asarray(mask)

      result = image.load_img(result_file)
      result = np.asarray(result) 

      pos_intersection = np.logical_and(mask, result)
      pos_union = np.logical_or(mask, result)
      pos_iou_score[n] = np.sum(pos_intersection) / np.sum(pos_union)

      neg_intersection = np.logical_and(1 - mask, 1- result)
      neg_union = np.logical_or(1-mask, 1- result)
      neg_iou_score[n] = np.sum(neg_intersection) / np.sum(neg_union)

      if np.isnan(pos_iou_score[n]): (pos_iou_score[n]) = neg_iou_score[n]

      avg_iou_sing[n] = ( pos_iou_score[n] + neg_iou_score[n] ) / 2
      #print(avg_iou_sing[n])

      TP = TP + np.sum(pos_intersection)
      TN = TN + np.sum(neg_intersection)
      FP = FP + np.sum(np.logical_and(1-mask, result))
      FN = FN + np.sum(np.logical_and(mask, 1-result))

      
  pos_avg_iou = np.mean(pos_iou_score)
  neg_avg_iou = np.mean(neg_iou_score)
  avg_iou = (pos_avg_iou+neg_avg_iou)/2

  print(avg_iou_sing)


  stand_deviation = np.std(avg_iou_sing)


  acc = (TP + TN)/(TP+TN+FP+FN)
  prec = TP/(TP+FP)
  recall = TP/(TP+FN)


  #print("positive avg iou:", pos_avg_iou)
  #print("negative avg iou:", neg_avg_iou)
  #print("avg iou:", avg_iou)
  #print("TP:", TP, "TN:", TN, "FP", FP, "FN", FN)
  #print("Acc:", acc, "Prec:", prec, "Recall:", recall)
  #print("std. dev. of avg iou:", stand_deviation)

  
  return avg_iou, avg_iou_sing, acc, stand_deviation

In [ ]:
# Predict Images in a Folder


%cd /content/
folder_path = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Firefront Dataset"
result_folder = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Firefront Dataset Results"
ids = next(os.walk(folder_path)) 
ids = ids[2]
print("No. of images = ", len(ids))

best_avg_iou = 0;
best_seg_threshold = 0;
best_class_threshold = 0;
best_ratio = 0;
best_size = 0;

min_patch_size = 400;

for class_threshold_ori in [0.4]:    #0.3
  for seg_threshold in [0.4]:            #0.4
    for ratio in [ 0.01 ]:  #0.00001 



        print ("class_threshold:",class_threshold_ori )
        print ("seg_threshold:",seg_threshold )
        print ("ratio:",ratio )
        
        print("\n############# Q+S ################\n")

        class_threshold = 0.0

        for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
          # Load images
          filename = folder_path+"/"+id_
          global_image, pred_bin = image_quad_prediction(filename, ratio, min_patch_size)
          #img_compressed, global_image, pred_bin = image_slicer_prediction (filename,100000)
          pred_bin.save(result_folder+"/"+id_)
                  
        qs_avg_iou, qs_avg_iou_sing, qs_acc, qs_stand_deviation = Calculate_Performance(0)

        print("\n############# C+S ################\n")

        class_threshold = class_threshold_ori

        for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
            # Load images
            filename = folder_path+"/"+id_
            #global_image, pred_bin = image_quad_prediction(filename)
            img_compressed, global_image, pred_bin = image_slicer_prediction (filename,100000)
            pred_bin.save(result_folder+"/"+id_)
                  
        cs_avg_iou, cs_avg_iou_sing, cs_acc, cs_stand_deviation = Calculate_Performance(0)

        print("\n############# S ################\n")
        class_threshold = 0.0

        for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
            # Load images
            filename = folder_path+"/"+id_
            #global_image, pred_bin = image_quad_prediction(filename)
            img_compressed, global_image, pred_bin = image_slicer_prediction (filename,100000)
            pred_bin.save(result_folder+"/"+id_)
                  
        s_avg_iou, s_avg_iou_sing, s_acc, s_stand_deviation = Calculate_Performance(0)

        print("\n############# Q+C+S ################\n")

        class_threshold = class_threshold_ori

        for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
            # Load images
            filename = folder_path+"/"+id_
            global_image, pred_bin = image_quad_prediction(filename, ratio, min_patch_size)
            #img_compressed, global_image, pred_bin = image_slicer_prediction (filename,100000)
            pred_bin.save(result_folder+"/"+id_)
                  
        qcs_avg_iou, qcs_avg_iou_sing, qcs_acc, qcs_stand_deviation = Calculate_Performance(0)

        data = (qcs_avg_iou_sing, qs_avg_iou_sing, cs_avg_iou_sing, s_avg_iou_sing)

        if qcs_avg_iou > best_avg_iou:
            best_avg_iou = qcs_avg_iou
            best_seg_threshold = seg_threshold
            best_class_threshold = class_threshold
            best_ratio = ratio
            best_size = min_patch_size

        print("\n#############################################\nQ+C+S:\n Avg.Iou: ",qcs_avg_iou, " SD: ", qcs_stand_deviation," Acc: ", qcs_acc )
        print("Q+S:\n Avg.Iou: ",qs_avg_iou, " SD: ", qs_stand_deviation," Acc: ", qs_acc )
        print("C+S:\n Avg.Iou: ",cs_avg_iou, " SD: ", cs_stand_deviation," Acc: ", cs_acc )
        print("S:\n Avg.Iou: ",s_avg_iou, " SD: ", s_stand_deviation," Acc: ", s_acc )

      
print("best avg_iou:", best_avg_iou )
print("best_seg_threshold:", best_seg_threshold )
print("best_class_threshold:", best_class_threshold )
print("best_ratio", best_ratio)

# Creating plot 
fig, ax = plt.subplots()

ax = fig.add_axes([0, 0, 1, 1]) 
bp = ax.boxplot(data )

plt.xlabel('Model')
plt.ylabel('Avg. Iou')
plt.title('Box Plot of Avg. IoU Distribution')
plt.show() 


In [ ]:
# Predict Images in a Folder


%cd /content/
folder_path = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Firefront Dataset"
result_folder = "/content/drive/My Drive/Gonçalo/IST/FIREFRONT/CNN/Firefront Dataset Smoke Results"
ids = next(os.walk(folder_path)) 
ids = ids[2]
print("No. of images = ", len(ids))

best_avg_iou = 0;
best_seg_threshold = 0;
best_class_threshold = 0;
best_ratio = 0;
best_size = 0;

min_patch_size = 700;

for class_threshold_ori in [0.4]:    #0.3
  for seg_threshold in [0.4]:            #0.4
    for ratio in [ 0.001 ]:  #0.00001 

#for class_threshold_ori in [0.3,0.4, 0.5, 0.6]:    #0.3
  #for seg_threshold in [0.3,0.4, 0.5, 0.6]:            #0.4
    #for ratio in [ 0.00001 , 0.0001 , 0.001]:  #0.00001
      #for min_patch_size in [ 200, 300, 400 ]:

        print ("class_threshold:",class_threshold_ori )
        print ("seg_threshold:",seg_threshold )
        print ("ratio:",ratio )

       
        class_threshold = class_threshold_ori

        for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
            # Load images
            filename = folder_path+"/"+id_
            global_image, pred_bin = image_quad_prediction(filename, ratio, min_patch_size)
            #img_compressed, global_image, pred_bin = image_slicer_prediction (filename,100000)
            pred_bin.save(result_folder+"/"+id_)
                  

     


/content
No. of images =  4
class_threshold: 0.0
seg_threshold: 0.8
ratio: 1e-08


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
#Predict A single Image
filename = "fogo_grande.jpg"
class_threshold = 0.4
seg_threshold = 0.4
global_image, pred_bin = image_quad_prediction(filename, 0.01, 4000)
pred_bin.save("result.png")
size = pred_bin.size
total_pixels = size[0]*size[1]
pred_bin = np.asarray(pred_bin)
ratio = (pred_bin/256).sum() / total_pixels
print("ratio= ",ratio*100)

ratio=  26.021320897547838


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0
